### Задание

1. Загрузите выборку weighted_knn.csv.
2. Реализуйте метод k ближайших соседей. В данной задаче будем использовать метрику Манхэттена:
$$ \rho(u,v)=\sum_{j=1}^{m}|u_j-v_j|.$$
3. Возьмите число соседей равным 5. Какого качества удается достичь на обучающей выборке?
4. Метрические методы довольно чувствительны к шумовым признакам. Исходную метрику можно модифицировать таким образом, чтобы каждый из признаков имел свой вес $\theta_j$:
$$ \rho(u,v)=\sum_{j=1}^{m}\theta_j|u_j-v_j|.$$
5. Реализуйте функцию, которая принимает на вход обучающую выборку `X`, `y`, число соседей `k` и веса признаков `theta`, а возвращает долю неверно классифицированных объектов при заданных параметрах.
6. С помощью функции `minimize` из пакета `scipy` найдите оптимальные значения параметров `theta`. Как изменилось качество классификации?

Буквально недавно (2 дня назад) выполнялась мною домашняя работа в параллельном курсе по Машинному обучению в ozon masters, в котором нужно было реализовать собственный KNN, поэтому мне повезло не тратить на это очень много времени. С реализацией и спефицикацией можно ознакомиться в папке knn.

In [1]:
from knn import KNNClassifier, BatchedKNNClassifier
from knn.distances import euclidean_distance, cosine_distance, manhattan_distance
from knn.model_selection import knn_cross_val_score
from knn.classification import BatchedMixin
from knn.nearest_neighbors import NearestNeighborsFinder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, BaseCrossValidator

import time
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2

# 1 

Загрузите выборку `weighted_knn.csv`.

In [3]:
data = pd.read_csv("weighted_knn.csv")

In [4]:
X = data.drop(columns="target").values
y = data.target.values
X.shape, y.shape

((500, 15), (500,))

In [5]:
n = 400
X_train, y_train, X_test, y_test = X[:n], y[:n], X[n:], y[n:]

In [6]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((400, 15), (400,), (100, 15), (100,))

# 2

Реализуйте метод `k` ближайших соседей. В данной задаче будем использовать метрику Манхэттена

In [7]:
metrics = ['euclidean', 'cosine', 'manhattan']
acc_score_result = {v: 0 for v in metrics}

# 3

Возьмите число соседей равным 5. Какого качества удается достичь на обучающей выборке?

In [8]:
for metric in acc_score_result.keys():
    knn_batch = BatchedKNNClassifier(n_neighbors=5, algorithm='my_own', metric=metric, W=False)
    knn_batch.set_batch_size(10)
    knn_batch.fit(X_train, y_train)
    y_pred = knn_batch.predict(X_test)
    acc_score_result[metric] = accuracy_score(y_pred, y_test)

In [9]:
acc_score_result

{'euclidean': 0.88, 'cosine': 0.88, 'manhattan': 0.91}

**Ответ**: 0.91

## 3.1 Кросс-валидация

In [10]:
k_list = np.arange(1, 11)
acc_score_cross_validation = {v: 0 for v in metrics}

In [11]:
for metric in acc_score_cross_validation.keys():
    print(f'Metric: {metric}')
    res = knn_cross_val_score(X, y, 
                              k_list=k_list,
                              scoring='accuracy',
                              cv=KFold(n_splits=10),
                              batch_size=1,
                              algorithm='my_own',
                              metric=metric)
    acc_score_cross_validation[metric] = res

Metric: euclidean
Metric: cosine
Metric: manhattan


In [12]:
cross_validation_result_accuracy_mean = {k: {r: np.mean(f) for r, f in acc.items()} for k, acc in acc_score_cross_validation.items()}

In [13]:
df_cross_validation_result_accuracy_mean = pd.DataFrame(cross_validation_result_accuracy_mean)
df_cross_validation_result_accuracy_mean

euclidean  cosine  manhattan
1       0.842   0.846      0.868
2       0.800   0.834      0.840
3       0.870   0.868      0.898
4       0.866   0.866      0.888
5       0.866   0.876      0.898
6       0.858   0.882      0.894
7       0.886   0.892      0.914
8       0.870   0.880      0.906
9       0.892   0.888      0.916
10      0.878   0.888      0.906

## 4. 

Метрические методы довольно чувствительны к шумовым признакам. Исходную метрику можно модифицировать таким образом, чтобы каждый из признаков имел свой вес $\theta_j$:
$$ \rho(u,v)=\sum_{j=1}^{m}\theta_j|u_j-v_j|.$$

Добавим `W = np.ndarray размерности D, где D - количество признаков`

In [14]:
d = np.ones(15)
assert d.shape[0] == X_train.shape[1]

In [15]:
d = np.array([100, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
d.shape
knn_batch = BatchedKNNClassifier(n_neighbors=5, algorithm='my_own', metric='manhattan', W=d)
knn_batch.set_batch_size(10)
knn_batch.fit(X_train, y_train)
y_pred = knn_batch.predict(X_test)
accuracy_score(y_pred, y_test)

0.91

# 5. 

Реализуйте функцию, которая принимает на вход обучающую выборку `X`, `y`, число соседей `k` и веса признаков `theta`, а возвращает долю неверно классифицированных объектов при заданных параметрах.

In [16]:
def get_test_error(w):
    knn_batch = BatchedKNNClassifier(n_neighbors=5, algorithm='my_own', metric='manhattan', 
                                     W = w)
    knn_batch.set_batch_size(10)
    knn_batch.fit(X_train, y_train)
    y_pred = knn_batch.predict(X_test)
    return 1 - accuracy_score(y_pred, y_test)

# 6

С помощью функции `minimize` из пакета `scipy` найдите оптимальные значения параметров `theta`. Как изменилось качество классификации?

In [17]:
res = scipy.optimize.minimize(get_test_error, x0=np.zeros(X_train.shape[1]))
res.x

array([ 3.35948153e-05,  2.55960498e-05,  5.59913589e-06,  2.39962967e-06,
        2.55960498e-05,  3.99938278e-06,  3.99938278e-06,  6.39901245e-06,
        7.99876556e-07,  1.59975311e-06, -3.19950622e-06,  7.99876556e-07,
        7.99876556e-06,  0.00000000e+00,  0.00000000e+00])

In [18]:
get_test_error(res.x)

0.07999999999999996